In [1]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, RepeatVector
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objects as go
from SleepClasses.AggregatedData import AggregatedData
import matplotlib.pyplot as plt

# Mostra tutte le colonne
pd.set_option('display.max_columns', None)

In [2]:
sleep_dfs_agg = pd.read_pickle(r'SleepData/sleep_dfs_agg.pkl')
sleep_dfs_gran = pd.read_pickle(r'SleepData/sleep_dfs_gran.pkl')
agg = AggregatedData(sleep_dfs_gran['2'], sleep_dfs_agg['2'])

df = agg.getAggregatedDataDf()

/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:594: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(merged_aggregated.values(), ignore_index=True)
/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:647: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modified_rows_df = pd.concat([modified_rows_df, df.loc[[idx]]])
/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:650: FutureWarning: The behavior of DataFrame concatenation with 

In [3]:
# Supponiamo che `df` sia il tuo DataFrame
measures = ["HrAverage", "HrMax", "HrMin", "RmssdAverage",
            "RmssdMax", "RmssdMin", "RrAverage", "RrMax", "RrMin", "Sdnn_1Average",
            "Sdnn_1Max", "Sdnn_1Min", "SnoringAverage", "SnoringMax", "SnoringMin",
            "LightSleepDuration", "RemSleepDuration", "DeepSleepDuration",
            "WakeUpDuration", "WakeUpCount", "ManualSleepDuration",
            "UndefinedSleepDuration", "OutOfBedCount", "OutOfBedTime",
            "TotalSleepTime"]

In [4]:
# Preprocessing: normalizzazione
scaler = MinMaxScaler()

# Normalizza il DataFrame
df[measures] = scaler.fit_transform(df[measures])

# Visualizza il DataFrame normalizzato
df

,Date,StartValue,EndValue,LightSleepDuration,DeepSleepDuration,RemSleepDuration,ManualSleepDuration,UndefinedSleepDuration,WakeUpDuration,TotalSleepTime,Nap,HrAverage,HrMax,HrMin,WakeUpCount,OutOfBedCount,model,model_id,RmssdAverage,RmssdMax,RmssdMin,RrAverage,RrMax,RrMin,Sdnn_1Average,Sdnn_1Max,Sdnn_1Min,SnoringAverage,SnoringMax,SnoringMin,OutOfBedTime,OutOfBed
0,2023-08-06,2023-08-05 21:42:00,2023-08-06 08:16:00,0.692157,1.000000,0.000000,0.0,0.0,0.138554,0.807428,False,0.000000,0.62,0.088889,0.500,0.000000,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,[]
1,2023-09-01,2023-09-01 00:24:00,2023-09-01 08:32:00,0.549020,0.280851,0.087558,0.0,0.0,0.322289,0.500688,False,0.166667,0.36,0.111111,0.125,0.166667,NaN,NaN,0.603774,0.339869,0.0,0.714286,0.774194,0.473684,0.528090,0.257009,0.0,0.111111,1.0,0.0,0.114286,"[{'from_value': 1, 'to_value': 0, 'next_value'..."
2,2023-09-01,2023-09-01 14:17:00,2023-09-01 15:59:00,0.003922,0.140426,0.009217,0.0,0.0,0.195783,0.049519,True,0.750000,0.62,0.222222,0.125,0.000000,NaN,NaN,0.377358,0.196078,0.0,0.809524,0.806452,0.473684,0.314607,0.172897,0.0,0.000000,0.0,0.0,0.000000,[]
3,2023-09-02,2023-09-02 00:45:00,2023-09-02 08:05:00,0.517647,0.263830,0.433180,0.0,0.0,0.060241,0.576341,False,0.055556,0.12,0.088889,0.000,0.000000,NaN,NaN,0.622642,0.287582,0.0,0.714286,0.838710,0.631579,0.595506,0.299065,0.0,0.055556,1.0,0.0,0.000000,[]
4,2023-09-02,2023-09-02 16:14:00,2023-09-02 19:36:00,0.147059,0.153191,0.193548,0.0,0.0,0.132530,0.209078,True,0.138889,0.18,0.177778,0.125,0.000000,NaN,NaN,0.849057,0.947712,0.0,0.761905,0.741935,0.473684,0.550562,0.252336,0.0,0.000000,0.0,0.0,0.000000,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,2024-11-10,2024-11-10 00:39:00,2024-11-10 07:33:00,0.592157,0.242553,0.129032,0.0,0.0,0.081325,0.530949,False,0.222222,0.44,0.177778,0.125,0.166667,32.0,63.0,0.905660,0.555556,0.0,0.761905,0.709677,0.526316,0.651685,0.742991,0.0,0.583333,1.0,0.0,0.040000,"[{'from_value': 1, 'to_value': 0, 'next_value'..."
651,2024-11-10,2024-11-10 13:23:00,2024-11-10 14:20:00,0.015686,0.063830,0.000000,0.0,0.0,0.102410,0.030261,True,0.916667,0.68,0.933333,0.000,0.000000,32.0,63.0,0.471698,0.366013,0.0,0.904762,0.806452,0.578947,0.393258,0.794393,0.0,0.000000,0.0,0.0,0.000000,[]
652,2024-11-11,2024-11-11 00:50:00,2024-11-11 08:16:00,0.443137,0.323404,0.207373,0.0,0.0,0.237952,0.475928,False,0.305556,0.40,0.222222,0.125,0.166667,32.0,63.0,0.849057,0.542484,0.0,0.761905,0.806452,0.526316,0.617978,0.962617,0.0,0.527778,1.0,0.0,0.131429,"[{'from_value': 3, 'to_value': 0, 'next_value'..."
653,2024-11-12,2024-11-11 23:08:00,2024-11-12 07:16:00,0.592157,0.157447,0.460829,0.0,0.0,0.147590,0.602476,False,0.194444,0.44,0.222222,0.125,0.166667,32.0,63.0,0.867925,0.529412,0.0,0.761905,0.774194,0.526316,0.573034,0.752336,0.0,0.166667,1.0,0.0,0.045714,"[{'from_value': 3, 'to_value': 0, 'next_value'..."


In [5]:
df['StartValue'] = pd.to_datetime(df['StartValue'])
df['EndValue'] = pd.to_datetime(df['EndValue'])

# Trova il primo giorno disponibile nel dataset
start_date = df['EndValue'].min()

# Calcola la differenza in giorni tra ogni data e la data iniziale
df['DaysSinceStart'] = (df['EndValue'] - start_date).dt.days

# Calcola il numero di settimane relative al primo giorno disponibile
df['WeekNumber'] = (df['DaysSinceStart'] // 7) + 1

# Aggiunta del primo e ultimo giorno della settimana come colonne
df['WeekStart'] = df.groupby('WeekNumber')['StartValue'].transform('min')
df['WeekEnd'] = df.groupby('WeekNumber')['EndValue'].transform('max')

# Raggruppa per settimana
weekly_data = df.groupby('WeekNumber').agg({
    'WeekStart':                'first',  # Usa il primo valore della settimana
    'WeekEnd':                  'first',    # Usa l'ultimo valore della settimana
    'HrAverage' :               ['mean', 'std', 'min', 'max'],
    'HrMax' :                   ['mean', 'std', 'min', 'max'],
    'HrMin' :                   ['mean', 'std', 'min', 'max'],
    'RmssdAverage' :            ['mean', 'std', 'min', 'max'],
    'RmssdMax' :                ['mean', 'std', 'min', 'max'],
    'RmssdMin' :                ['mean', 'std', 'min', 'max'],
    'RrAverage' :               ['mean', 'std', 'min', 'max'],
    'RrMax' :                   ['mean', 'std', 'min', 'max'],
    'RrMin' :                   ['mean', 'std', 'min', 'max'],
    'Sdnn_1Average' :           ['mean', 'std', 'min', 'max'],
    'Sdnn_1Max' :               ['mean', 'std', 'min', 'max'],
    'Sdnn_1Min' :               ['mean', 'std', 'min', 'max'],
    'SnoringAverage' :          ['mean', 'std', 'min', 'max'],
    'SnoringMax' :              ['mean', 'std', 'min', 'max'],
    'SnoringMin' :              ['mean', 'std', 'min', 'max'],
    'LightSleepDuration' :      ['mean', 'std', 'min', 'max'],
    'RemSleepDuration' :        ['mean', 'std', 'min', 'max'],
    'DeepSleepDuration' :       ['mean', 'std', 'min', 'max'],
    'WakeUpDuration' :          ['mean', 'std', 'min', 'max'],
    'WakeUpCount' :             ['mean', 'std', 'min', 'max'],
    'ManualSleepDuration' :     ['mean', 'std', 'min', 'max'],
    'UndefinedSleepDuration' :  ['mean', 'std', 'min', 'max'],
    'OutOfBedCount' :           ['mean', 'std', 'min', 'max'],
    'OutOfBedTime' :            ['mean', 'std', 'min', 'max'],
    'TotalSleepTime':           ['mean', 'std', 'min', 'max'],
    'Nap':                      'sum'  # Conta i pisolini in ogni settimana
}).reset_index()
columnList = ['WeekNumber',
                'WeekStart', 'WeekEnd',
                'MEAN_HrAverage', 'STD_HrAverage', 'MIN_HrAverage', 'MAX_HrAverage',
                'MEAN_HrMax', 'STD_HrMax', 'MIN_HrMax', 'MAX_HrMax',
                'MEAN_HrMin', 'STD_HrMin', 'MIN_HrMin', 'MAX_HrMin',
                'MEAN_RmssdAverage', 'STD_RmssdAverage', 'MIN_RmssdAverage', 'MAX_RmssdAverage',
                'MEAN_RmssdMax','STD_RmssdMax','MIN_RmssdMax','MAX_RmssdMax',
                'MEAN_RmssdMin','STD_RmssdMin','MIN_RmssdMin','MAX_RmssdMin',
                'MEAN_RrAverage','STD_RrAverage','MIN_RrAverage','MAX_RrAverage',
                'MEAN_RrMax','STD_RrMax','MIN_RrMax','MAX_RrMax',
                'MEAN_RrMin','STD_RrMin','MIN_RrMin','MAX_RrMin',
                'MEAN_Sdnn_1Average','STD_Sdnn_1Average','MIN_Sdnn_1Average','MAX_Sdnn_1Average',
                'MEAN_Sdnn_1Max','STD_Sdnn_1Max','MIN_Sdnn_1Max','MAX_Sdnn_1Max',
                'MEAN_Sdnn_1Min','STD_Sdnn_1Min','MIN_Sdnn_1Min','MAX_Sdnn_1Min',
                'MEAN_SnoringAverage','STD_SnoringAverage','MIN_SnoringAverage','MAX_SnoringAverage',
                'MEAN_SnoringMax','STD_SnoringMax','MIN_SnoringMax','MAX_SnoringMax',
                'MEAN_SnoringMin','STD_SnoringMin','MIN_SnoringMin','MAX_SnoringMin',
                'MEAN_LightSleepDuration','STD_LightSleepDuration','MIN_LightSleepDuration','MAX_LightSleepDuration',
                'MEAN_RemSleepDuration','STD_RemSleepDuration','MIN_RemSleepDuration','MAX_RemSleepDuration',
                'MEAN_DeepSleepDuration','STD_DeepSleepDuration','MIN_DeepSleepDuration','MAX_DeepSleepDuration',
                'MEAN_WakeUpDuration','STD_WakeUpDuration','MIN_WakeUpDuration','MAX_WakeUpDuration',
                'MEAN_WakeUpCount','STD_WakeUpCount','MIN_WakeUpCount','MAX_WakeUpCount',
                'MEAN_ManualSleepDuration','STD_ManualSleepDuration','MIN_ManualSleepDuration','MAX_ManualSleepDuration',
                'MEAN_UndefinedSleepDuration','STD_UndefinedSleepDuration','MIN_UndefinedSleepDuration','MAX_UndefinedSleepDuration',
                'MEAN_OutOfBedCount','STD_OutOfBedCount','MIN_OutOfBedCount','MAX_OutOfBedCount',
                'MEAN_OutOfBedTime','STD_OutOfBedTime','MIN_OutOfBedTime','MAX_OutOfBedTime',
                'MEAN_TotalSleepTime', 'STD_TotalSleepTime', 'MIN_TotalSleepTime', 'MAX_TotalSleepTime',
                'SUM_Nap']
# Rinomina colonne per facilità di utilizzo
weekly_data.columns = columnList

# Aggiunta di una colonna con il range di date della settimana
weekly_data['DateRange'] = weekly_data['WeekStart'].dt.strftime('%d %b %Y') + ' - ' + weekly_data['WeekEnd'].dt.strftime('%d %b %Y')

# Imposta la colonna 'std' a 0 nei casi in cui abbia NaN
columns_with_std = [col for col in weekly_data.columns if 'std' in col.lower()]  # Trova tutte le colonne 'std'

#ci sono casi in cui la standard deviation non si pò fare perchè la persona ha dormito solo 1 volta nella settimana, quindi metto a 0 la std
for col in columns_with_std:
    weekly_data[col] = weekly_data[col].fillna(0)  # Riempie i NaN solo nelle colonne 'std's

In [6]:
# Funzione per addestrare un autoencoder LSTM
def train_autoencoder(X_train):
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))  # Definisci la forma in modo esplicito
    encoded = LSTM(64, activation='relu')(input_layer)
    decoded = RepeatVector(X_train.shape[1])(encoded)
    decoded = LSTM(X_train.shape[2], activation='sigmoid', return_sequences=True)(decoded)
    
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)
    
    return autoencoder

In [7]:
# Funzione per preparare i dati per LSTM (convertire il dataframe in sequenze temporali)
def prepare_data(df, time_steps=1):
    X, y = [], []
    for i in range(len(df) - time_steps):
        X.append(df.iloc[i:i+time_steps].values)
        y.append(df.iloc[i+time_steps].values)
    return np.array(X), np.array(y)

In [8]:
data_normalized = weekly_data.drop(columns=['WeekNumber', 'WeekStart', 'WeekEnd', 'DateRange']).values
data_normalized = data_normalized.astype('float64')

In [9]:
# Preparazione dei dati per LSTM
time_steps = 4  # Numero di settimane da usare per fare la previsione
X, y = prepare_data(pd.DataFrame(data_normalized), time_steps)

In [10]:
# Dividere i dati in train e test
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

In [11]:
# Reshape dei dati in (numero_settimane, 1, numero_feature)
X = data_normalized.reshape(data_normalized.shape[0], 1, data_normalized.shape[1])
# Addestramento iniziale con le prime 4 settimane
initial_data = X[:4]
autoencoder = train_autoencoder(initial_data)  # Usa solo le prime 4 settimane per addestrare

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.1154 - val_loss: 0.2079
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1151 - val_loss: 0.2074
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1147 - val_loss: 0.2068
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1144 - val_loss: 0.2062
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1140 - val_loss: 0.2055
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1136 - val_loss: 0.2048
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1132 - val_loss: 0.2041
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1127 - val_loss: 0.2033
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1122 - val_loss: 0.2024
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1117 - val_loss: 0.2014
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1112 - val_loss: 0.2004
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1105 - val_loss: 0.1992
Epo

In [12]:
# Funzione per calcolare l'errore di ricostruzione
def calculate_reconstruction_error(X, autoencoder):
    X_pred = autoencoder.predict(X)
    # Calcoliamo l'errore per ogni feature
    reconstruction_error = np.mean(np.abs(X - X_pred), axis=(2))  # Errori lungo axis 2 (features)
    return reconstruction_error

In [13]:
# Rilevamento anomalie con errore di ricostruzione
errors = []
anomalous_points = []  # Lista per memorizzare gli indici dei punti anomali
for i in range(5, len(X)):
    reconstruction_error = calculate_reconstruction_error(X[i:i+1], autoencoder)
    errors.append(reconstruction_error[0][0])
    
    # Se l'errore di ricostruzione supera la soglia, considera l'episodio come anomalo
    if reconstruction_error > np.percentile(errors, 95):
        print(f'Settimana {i} è anomala')
        anomalous_points.append(i)  # Aggiungi l'indice dell'anomalia
    # Aggiungi il nuovo dato per aggiornare il modello (addestramento incrementale)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Settimana 10 è anomala
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Settimana 11 è anomala
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Settimana 13 è anomala
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Settimana 21 è anomala
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Settimana 25 è anomala
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9m

In [14]:
# Creiamo il grafico con Plotly
fig = go.Figure()

# Aggiungiamo il grafico dell'errore di ricostruzione
fig.add_trace(go.Scatter(
    x=list(range(len(errors))),
    y=errors,
    mode='lines',
    name='Errore di Ricostruzione',
    line=dict(color='blue')
))

# Aggiungiamo una linea di soglia (per esempio al 95° percentile dell'errore)
threshold_value = np.percentile(errors, 95)
fig.add_trace(go.Scatter(
    x=[0, len(errors)-1],
    y=[threshold_value, threshold_value],
    mode='lines',
    name='Soglia Anomalia',
    line=dict(color='red', dash='dash')
))

# Aggiungi i punti anomali
fig.add_trace(go.Scatter(
    x=anomalous_points,
    y=[errors[i] for i in anomalous_points],
    mode='markers',
    name='Punti Anomali',
    marker=dict(color='red', size=10, symbol='circle')
))

# Layout del grafico
fig.update_layout(
    title='Errore di Ricostruzione Settimanale',
    xaxis_title='Settimana',
    yaxis_title='Errore di Ricostruzione',
    showlegend=True
)

# Mostra il grafico
fig.show()